#### Unset common proxy environment variables

In [1]:
import os

proxy_vars = [
    'http_proxy',
    'https_proxy',
    'ftp_proxy',
    'all_proxy',
    'HTTP_PROXY', 
    'HTTPS_PROXY',
    'FTP_PROXY',
    'ALL_PROXY',
    'SOCKS_PROXY',
    'socks_proxy'
]

for var in proxy_vars:
    if var in os.environ:
        del os.environ[var]

In [2]:
from langchain_core.documents import Document

In [3]:
# Create example documents about pets with metadata
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [5]:
type(documents[0])

langchain_core.documents.base.Document

## PDF Loader


In [6]:
from langchain_community.document_loaders import PyPDFLoader

In [7]:
file_path = "pdf/nke-10k-2023.pdf"

loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


In [8]:
docs[0].metadata

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0',
 'creator': 'EDGAR Filing HTML Converter',
 'creationdate': '2023-07-20T16:22:00-04:00',
 'title': '0000320187-23-000039',
 'author': 'EDGAR Online, a division of Donnelley Financial Solutions',
 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31',
 'keywords': '0000320187-23-000039; ; 10-K',
 'moddate': '2023-07-20T16:22:08-04:00',
 'source': 'pdf/nke-10k-2023.pdf',
 'total_pages': 107,
 'page': 0,
 'page_label': '1'}

In [10]:
print(docs[0].page_content)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE FISCAL YEAR ENDED MAY 31, 2023
OR
☐  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE TRANSITION PERIOD FROM                         TO                         .
Commission File No. 1-10635
NIKE, Inc.
(Exact name of Registrant as specified in its charter)
Oregon 93-0584541
(State or other jurisdiction of incorporation) (IRS Employer Identification No.)
One Bowerman Drive, Beaverton, Oregon 97005-6453
(Address of principal executive offices and zip code)
(503) 671-6453
(Registrant's telephone number, including area code)
SECURITIES REGISTERED PURSUANT TO SECTION 12(B) OF THE ACT:
Class B Common Stock NKE New York Stock Exchange
(Title of each class) (Trading symbol) (Name of each exchange on which registered)
SECURITIES REGISTERED PURSU

## Text Splitter

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

## Embeddings

In [13]:
from langchain_ollama import OllamaEmbeddings

In [19]:
# 远程Ollama服务ip
import platform
REMOTE_HOST = "192.168.11.8" if platform.system() == "Linux" else "127.0.0.1"
# Ollama服务端口
OLLAMA_PORT = 11434
base_url=f"http://{REMOTE_HOST}:{OLLAMA_PORT}"
base_url

'http://192.168.11.8:11434'

In [20]:
embeddings = OllamaEmbeddings(model="deepseek-r1:8b", base_url=base_url)

In [21]:
embeddings

OllamaEmbeddings(model='deepseek-r1:8b', base_url='http://192.168.11.8:11434', client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [33]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 4096

[-0.0003399349, -0.023435304, -0.0019291437, 0.0007058404, -0.00543917, -0.022657108, 0.0072528343, -0.009554426, 0.0035034816, 0.009167676]


In [34]:
len(vector_1)

4096

## Vector stores

In [35]:
from langchain_core.vectorstores import InMemoryVectorStore

In [36]:
vector_store = InMemoryVectorStore(embeddings)

In [37]:
ids = vector_store.add_documents(documents=all_splits)
len(ids)

516

In [43]:
ids[:5]

['2ef43ee0-73d5-4a05-a4d3-6da5aca1263a',
 '7c05bc1f-7ec3-4e55-affa-d8ff2bd2d6c6',
 'b6c25a0b-7b00-48b5-a7ce-fd9d3a22a6a6',
 'd6d22fa2-db2e-48e3-ad90-0da6c0f77ca2',
 'e37cb167-de4f-4683-9222-9d133ca00fae']

In [46]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='polyester and natural fiber textiles and polyurethane films — are locally available to manufacturers. The principal materials used in our apparel products — natural and
synthetic fabrics, yarns and threads (both virgin and recycled), specialized performance fabrics designed to efficiently wick moisture away from the body, retain heat and
repel rain and/or snow as well as plastic and metal hardware — are also available in countries where our manufacturing takes place. Both our apparel and footwear
products are dependent upon the ability of our contract manufacturers to locate, train, employ and retain adequate personnel. NIKE contract manufacturers and materials
suppliers buy raw materials and are subject to wage rates and other labor standards that are oftentimes regulated by the governments of the countries in which our
products are manufactured.' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-

In [49]:
print(results[0].metadata.keys())

dict_keys(['producer', 'creator', 'creationdate', 'title', 'author', 'subject', 'keywords', 'moddate', 'source', 'total_pages', 'page', 'page_label', 'start_index'])


In [47]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='approximate amounts of base compensation and minimum guaranteed royalty fees we are obligated to pay athlete, public figure, sport team and league
endorsers of our products. Actual payments under some contracts may be higher than these amounts as these contracts provide for bonuses to be paid to the
endorsers based upon athletic achievements and/or royalties on product sales in future periods. Actual payments under some contracts may also be lower as
these contracts include provisions for reduced payments if athletic performance declines in future periods. In addition to the cash payments, we are obligated to
furnish our endorsers with NIKE product for their use. It is not possible to determine how much we will spend on this product on an annual basis as the amount
of product provided to the endorsers will depend on many factors and the contracts generally do not stipulate a minimum amount of cash to be spent on the
product.' metadata={'producer': 'EDGRpdf Service w/ EO.P

In [50]:
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.39230263594464876

page_content='Table of Contents
NORTH AMERICA
(Dollars in millions) FISCAL 2023 FISCAL 2022 % CHANGE
% CHANGEEXCLUDINGCURRENCYCHANGESFISCAL 2021 % CHANGE
% CHANGEEXCLUDINGCURRENCYCHANGES
Revenues by:
Footwear $ 14,897 $ 12,228 22 % 22 %$ 11,644 5 % 5 %
Apparel 5,947 5,492 8 % 9 % 5,028 9 % 9 %
Equipment 764 633 21 % 21 % 507 25 % 25 %
TOTAL REVENUES $ 21,608 $ 18,353 18 % 18 %$ 17,179 7 % 7 %
Revenues by:    
Sales to Wholesale Customers $ 11,273 $ 9,621 17 % 18 %$ 10,186 -6 % -6 %
Sales through NIKE Direct 10,335 8,732 18 % 18 % 6,993 25 % 25 %
TOTAL REVENUES $ 21,608 $ 18,353 18 % 18 %$ 17,179 7 % 7 %
EARNINGS BEFORE INTEREST AND TAXES $ 5,454 $ 5,114 7 % $ 5,089 0 %
FISCAL 2023 COMPARED TO FISCAL 2022
• North America revenues increased 18% on a currency-neutral basis, primarily due to higher revenues in Men's and the Jordan Brand. NIKE Direct revenues
increased 18%, driven by strong digital sales growth of 23%, comparable store sales growth of 9% and the 

In [51]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
NORTH AMERICA
(Dollars in millions) FISCAL 2023 FISCAL 2022 % CHANGE
% CHANGEEXCLUDINGCURRENCYCHANGESFISCAL 2021 % CHANGE
% CHANGEEXCLUDINGCURRENCYCHANGES
Revenues by:
Footwear $ 14,897 $ 12,228 22 % 22 %$ 11,644 5 % 5 %
Apparel 5,947 5,492 8 % 9 % 5,028 9 % 9 %
Equipment 764 633 21 % 21 % 507 25 % 25 %
TOTAL REVENUES $ 21,608 $ 18,353 18 % 18 %$ 17,179 7 % 7 %
Revenues by:    
Sales to Wholesale Customers $ 11,273 $ 9,621 17 % 18 %$ 10,186 -6 % -6 %
Sales through NIKE Direct 10,335 8,732 18 % 18 % 6,993 25 % 25 %
TOTAL REVENUES $ 21,608 $ 18,353 18 % 18 %$ 17,179 7 % 7 %
EARNINGS BEFORE INTEREST AND TAXES $ 5,454 $ 5,114 7 % $ 5,089 0 %
FISCAL 2023 COMPARED TO FISCAL 2022
• North America revenues increased 18% on a currency-neutral basis, primarily due to higher revenues in Men's and the Jordan Brand. NIKE Direct revenues
increased 18%, driven by strong digital sales growth of 23%, comparable store sales growth of 9% and the addition of new stores.' met